In [1]:
!pip install /home/ma-user/work/LSTM-GAF/mindnlp-0.1.1-py3-none-any.whl

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
Processing /home/ma-user/work/step_into_chatgpt/mindnlp-0.1.1-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 11.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.6/36.6 MB 39.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 984.9/984.9 kB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.7/96.7 kB 25.2 MB/s eta 0:00:00
     ━━━━

In [2]:
!pip install regex

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.7/757.7 kB 38.9 MB/s eta 0:00:00
DEPRECATION: moxing-framework 2.1.7.dc1f3d0b has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of moxing-framework or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [14]:
from mindspore.ops import functional as F, composite as C, operations as P
_atan_opt = C.MultitypeFuncGraph("atan_opt")
map=C.Map()
@_atan_opt.register("Tensor")
def atan_p(x):
    a=P.Atan()
    return 0.2*a(x*10)
class sgd_act(nn.SGD):
    
    def construct(self,g):
        
        g=map(_atan_opt,g)
        return super().construct(g)

In [ ]:

import mindspore as ms
import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn, context
from mindnlp.transforms import PadTransform
from mindnlp.transforms.tokenizers import BertTokenizer
from mindnlp.engine import Trainer, Evaluator
from mindnlp.engine.callbacks import CheckpointCallback, BestModelCallback
from mindnlp.metrics import Accuracy
import os
from mindspore import ops, ms_function
from mindspore.amp import init_status, all_finite, DynamicLossScaler
import matplotlib
import matplotlib.pyplot as plt
import time
import numpy as np
from mindspore.train.serialization import save_checkpoint
#from tqdm import tqdm
from mindnlp.models import BertForSequenceClassification
from mindnlp._legacy.amp import auto_mixed_precision
from mindnlp.modules import Accumulator
#import GAF
from mindnlp.models import BertModel
import mindnlp.transforms.tokenizers as t
import pandas as pd
import json
import moxing as mox
from mindspore.communication.management import init, get_local_rank_size
data_dir='/home/ma-user/work/LSTM-GAF/'
model_dir='/home/ma-user/work/LSTM-GAF/model'
class R8Dataset:
    def __init__(self, path):
        self.path = path
        self._labels, self._text_a = [], []
        self._load()
    def _load(self):
        self.df = pd.read_csv(self.path)
        labels ={'ship': 0,
                  'money-fx': 1,
                  'grain': 2,
                  'acq': 3,
                  'trade': 4,
                  'earn': 5,
                  'crude': 6,
                  'interest': 7}
        self.df['intent'] = [labels[i] for i in self.df['intent']]

    def __getitem__(self, index):
        return self.df['text'][index],self.df['intent'][index]

    def __len__(self):
        return len(self.df)

def process_dataset(source, tokenizer,pad_value, batch_size=32,shuffle=True):
    column_names = ["text_a","label"]
    rename_columns = ["input_ids","label"]

    dataset = GeneratorDataset(source, column_names=column_names, shuffle=shuffle)
    # transforms
    pad_op = PadTransform(64, pad_value=pad_value)
    type_cast_op = transforms.TypeCast(mindspore.int32)

    # map dataset
    dataset = dataset.map(operations=[tokenizer, pad_op], input_columns="text_a")
    dataset = dataset.map(operations=[type_cast_op], input_columns="label")
    # rename dataset
    dataset = dataset.rename(input_columns=column_names, output_columns=rename_columns)
    # batch dataset
    dataset = dataset.batch(batch_size)

    return dataset

tokenizer = BertTokenizer.from_pretrained(data_dir)
pad_value = tokenizer.token_to_id('[PAD]')
dataset_train = process_dataset(R8Dataset(data_dir + "/r8/r8-train-stemmed.csv"),
                                tokenizer, pad_value)
dataset_val = process_dataset(R8Dataset(data_dir + "/r8/r8-dev-stemmed.csv"), tokenizer,
                              pad_value)
print(dataset_val.get_dataset_size())
dataset_test = process_dataset(R8Dataset(data_dir + "/r8/r8-test-stemmed.csv"), tokenizer,
                               pad_value, shuffle=False)
gaf = False
# set bert config and define parameters for training
reslut_record = {'loss': [], 'acc': []}
total = dataset_train.get_dataset_size()
loss_list = []
acc_list=[]
model = BertForSequenceClassification.from_pretrained(model_dir)
model = auto_mixed_precision(model, 'O1')
loss_fn = nn.CrossEntropyLoss()
# n_model=BertForSequenceClassificationWithLoss(model,loss_fn)

optimizer = sgd_act(model.trainable_params(),
              learning_rate=1e-4,weight_decay=1e-8)  #



metric = Accuracy()
'''# define callbacks to save checkpoints
ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='bert_emotect', epochs=1, keep_checkpoint_max=2)
best_model_cb = BestModelCallback(save_path='checkpoint', ckpt_name='bert_emotect_best', auto_load=True)
trainer = Trainer(network=model, train_dataset=dataset_train,
                  eval_dataset=dataset_val, metrics=metric,
                  epochs=5, loss_fn=loss_fn, optimizer=optimizer, callbacks=[ckpoint_cb, best_model_cb],
                  jit=False)'''

loss_scaler = DynamicLossScaler(scale_value=2 ** 10, scale_factor=2, scale_window=1000)
def forward_fn(inputs, labels):
    logits = model(inputs, labels=labels)
    loss = logits[0]
    return loss_scaler.scale(loss)
grad_fn = ops.value_and_grad(forward_fn, None, model.trainable_params())
@ms_function
def train_step(data, label):
    status = init_status()
    data = ops.depend(data, status)
    loss, grads = grad_fn(data, label)
    loss = loss_scaler.unscale(loss)
    is_finite = all_finite(grads, status)
    if is_finite:
        grads = loss_scaler.unscale(grads)
        loss = ops.depend(loss, optimizer(grads))
    loss = ops.depend(loss, loss_scaler.adjust(is_finite))
    return loss, is_finite
for epoch in range(100):
        print(f'=========epoch:{epoch}============')
        loss_total = 0
        loss_100 = 0
        cur_step_nums = 0
        for batch_idx, (input_ids, labels) in enumerate(dataset_train.create_tuple_iterator()):
            cur_step_nums += 1
            loss, is_finite = train_step(input_ids, labels)
            loss_total += loss
            loss_100 += float(loss)
            if cur_step_nums % 25 == 0:
                print(loss_100/100)
                loss_list.append(loss_100 / 25)
                loss_100 = 0
        if epoch % 9 == 0:
            save_checkpoint(model, model_dir+f'/mindspore_{epoch}.ckpt')
            evaluator = Evaluator(network=model, eval_dataset=dataset_test, metrics=metric)
            evaluator.run(tgt_columns="label")
            acc_list.append(evaluator.metrics[0].eval())
reslut_record['loss'].append(loss_list)
reslut_record['acc'].append(acc_list)
with open(save_dir+f'/result_{device_id}.json','w+') as f:
    json.dump(reslut_record,fp=f,ensure_ascii=False,indent=True)
    f.close()

18


[WARNING] ME(1401:281472972290624,MainProcess):2023-09-22-22:02:38.760.437 [/home/ma-user/anaconda3/envs/MindSpore/lib/python3.7/site-packages/mindnlp/abc/models/pretrained_model.py:454] The following parameters in checkpoint files are not loaded:
['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.layer_norm.gamma', 'cls.predictions.transform.layer_norm.beta', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']


=========epoch:0============
0.5205530393123626
0.44863426327705386
0.397638418674469
0.3755353271961212
0.33423362374305726
0.3233847439289093


  0%|          | 0/69 [00:00<?, ?it/s]

Evaluate Score: {'Accuracy': 0.49520328917313844}
=========epoch:1============
0.325175541639328
0.3104212188720703
0.29575618863105774
0.31159837782382965
0.29560320734977724
0.2857597780227661
=========epoch:2============
0.28015927255153655
0.2811145275831223
0.26284232795238494
0.2636472362279892
0.2581382739543915
0.259300012588501
=========epoch:3============
0.26647972881793974
0.24107673346996308
0.24301410019397734
0.22896634817123412
0.23856590032577515
0.2356208771467209
=========epoch:4============
0.22525591373443604
0.23608627796173096
0.21688064008951188
0.22646700143814086
0.22526592910289764
0.2018087837100029
=========epoch:5============
0.2232166337966919
0.19870789736509323
0.20730115950107575
0.2250705873966217
0.19468271911144255
0.1971115654706955
=========epoch:6============
0.2186087363958359
0.1963529270887375
0.18586782187223436
0.1814999082684517
0.17573222309350967
0.1890942507982254
=========epoch:7============
0.17506281644105912
0.18307737410068511
0.166

...

Evaluate Score: {'Accuracy': 0.8227501142074006}
=========epoch:19============
0.09201083317399025
0.10459264561533928
0.09189464822411537
0.09329959750175476
0.09319598123431205
0.08710518106818199
=========epoch:20============
0.10129611164331437
0.0924319438636303
0.09096315614879132
0.09564982697367669
0.07968158811330796
0.09081619091331959
=========epoch:21============
0.08243395447731018
0.08666064485907554
0.08893999010324478
0.08084608994424343
0.09543333515524864
0.09570482268929481
=========epoch:22============
0.08102654531598091
0.07608182586729527
0.09766512006521225
0.08821096673607826
0.08700816839933395
0.07878241539001465
=========epoch:23============
0.0848123300075531
0.08261286668479442
0.08525145456194877
0.08170128628611564
0.07792502321302891
0.07919002868235112
=========epoch:24============
0.07974667340517044
0.08005424834787846
0.07855230145156383
0.0788246163725853
0.08131240405142308
0.07424125656485557
=========epoch:25============
0.07945282950997352
0.07

  0%|          | 0/69 [00:00<?, ?it/s]

Evaluate Score: {'Accuracy': 0.8433074463225217}
=========epoch:28============
0.07229872822761535
0.06552756786346435
0.07591427892446517
0.06774712517857552
0.0702089174091816
0.06854653224349022
=========epoch:29============
0.06651594735682011
0.05678108669817448
0.07416611656546593
0.07398310042917729
0.06762426383793355
0.060893437564373015
=========epoch:30============
0.06721452236175537
0.06655570216476918
0.06859348863363265
0.06436977177858352
0.06226745702326298
0.05429727517068386
=========epoch:31============
0.06230678424239158
0.061963842511177064
0.0697308398783207
0.06386150725185871
0.05963396109640598
0.05562817499041557
=========epoch:32============
0.07260732218623162
0.053814191073179245
0.06293557547032833
0.05695360168814659
0.05394873097538948
0.05842666737735271
=========epoch:33============
0.06918751329183578
0.05164087191224098
0.05204365000128746
0.05298761658370495
0.0617214473336935
0.05730623915791511
=========epoch:34============
0.054529949501156805


  0%|          | 0/69 [00:00<?, ?it/s]

Evaluate Score: {'Accuracy': 0.8684330744632253}
=========epoch:37============
0.047504742741584775
0.05417174369096756
0.04671931676566601
0.04468892768025398
0.04989541135728359
0.047774840518832205
=========epoch:38============
0.05300253294408321
0.03979097083210945
0.048332839459180835
0.03895648777484894
0.05308430276811123
0.042972169369459155
=========epoch:39============
0.04095973938703537
0.05203658923506737
0.04655785501003265
0.04290835089981556
0.042238963171839713
0.040756244882941245
=========epoch:40============
0.04100255779922009
0.04693788088858128
0.038446672335267065
0.053213845267891885
0.03375357381999493
0.0416313299536705
=========epoch:41============
0.041718261912465096
0.03995342247188091
0.038630761355161664
0.0379414252191782
0.04695342749357224
0.03748010583221913
=========epoch:42============
0.039875781014561656
0.036164589747786524
0.043148998469114304
0.03817580729722977
0.036594219729304316
0.03591857522726059
=========epoch:43============
0.0407163